Notebook stores the processed experimental data in SQLite to explore performance, efficiency, and variability.

In [1]:
import sqlite3
import pandas as pd

df = pd.read_csv("../data/processed/honey_pollen_efficiency_clean.csv")

conn = sqlite3.connect("../data/bee_honey.db")
df.to_sql("hive_trials", conn, if_exists="replace", index=False)
conn.close()


/home/ldmonte/workspace/SQLite/Bee_trials/bee-honey-efficiency-analysis/venv/lib/python3.12/site-packages/numpy/_core/getlimits.py:552: UserWarning: Signature b'\x00\xd0\xcc\xcc\xcc\xcc\xcc\xcc\xfb\xbf\x00\x00\x00\x00\x00\x00' for <class 'numpy.longdouble'> does not match any known type: falling back to type probe function.
This warnings indicates broken support for the dtype!
  machar = _get_machar(dtype)


In [2]:
conn = sqlite3.connect("../data/bee_honey.db")

query = """
SELECT hive_id,
       AVG(efficiency) AS avg_efficiency
FROM hive_trials
GROUP BY hive_id
ORDER BY avg_efficiency DESC;
"""

pd.read_sql(query, conn)


,hive_id,avg_efficiency
0,TML BRO 16.2.2,1154.166667
1,TML BRO 15.5.5,1070.833333
2,TML BRO 16.2.1,1052.083333
3,TML BRO 16.1.4,1025.000000
4,TML BRO 15.6.4,1006.666667
...,...,...
102,TML AMP 15.3.1,207.916667
103,TML AMP 15.2.2,190.000000
104,TML AMP 15.2.1,137.708333
105,LAO BRO 20.5.5,106.250000


In [3]:
query = """
SELECT
    COUNT(*) AS total_trials,
    AVG(pollen_input_g) AS avg_input,
    AVG(honey_output_g) AS avg_output,
    AVG(efficiency) AS avg_efficiency,
    AVG(honey_per_hour) AS avg_output_rate
FROM hive_trials;
"""

pd.read_sql(query, conn)



,total_trials,avg_input,avg_output,avg_efficiency,avg_output_rate
0,107,2344.672897,1.532264e+06,643.176372,31922.157321


In [4]:
query = """
SELECT
    CAST(pollen_input_g / 500 AS INT) * 500 AS input_bucket,
    COUNT(*) AS n_trials,
    AVG(efficiency) AS avg_efficiency,
    AVG(efficiency * efficiency) - AVG(efficiency) * AVG(efficiency) AS variance
FROM hive_trials
GROUP BY input_bucket
ORDER BY input_bucket;
"""

df_sql = pd.read_sql(query, conn)
df_sql

,input_bucket,n_trials,avg_efficiency,variance
0,0,2,73.333333,1083.506944
1,500,1,486.250000,0.000000
2,1500,3,700.625000,11459.375000
3,2000,94,661.616544,38058.863036
4,3000,7,556.160714,6448.513233


In [5]:
conn.close()
